In [1]:
import pandas as pd

# Import libraries
from sklearn.model_selection import cross_val_score, train_test_split
from scipy.stats import linregress
from tqdm import tqdm
import glob
import lightgbm as lgb
import numpy as np  # linear algebra
import optuna as op
import os
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import log_loss
from xgboost import XGBClassifier

In [2]:
!ls -GFlash ../input/march-machine-learning-mania-2024/

total 143M
   0 drwxr-xr-x 2 nobody    0 Mar  6 18:06 ./
4.0K drwxr-xr-x 3 root   4.0K Mar 15 20:01 ../
4.0K -rw-r--r-- 1 nobody 1.4K Mar  6 18:06 2024_tourney_seeds.csv
 12K -rw-r--r-- 1 nobody 9.1K Mar  6 18:06 Cities.csv
4.0K -rw-r--r-- 1 nobody 1.7K Mar  6 18:06 Conferences.csv
160K -rw-r--r-- 1 nobody 160K Mar  6 18:06 MConferenceTourneyGames.csv
2.5M -rw-r--r-- 1 nobody 2.5M Mar  6 18:06 MGameCities.csv
110M -rw-r--r-- 1 nobody 110M Mar  6 18:06 MMasseyOrdinals.csv
 72K -rw-r--r-- 1 nobody  72K Mar  6 18:06 MNCAATourneyCompactResults.csv
132K -rw-r--r-- 1 nobody 129K Mar  6 18:06 MNCAATourneyDetailedResults.csv
 16K -rw-r--r-- 1 nobody  15K Mar  6 18:06 MNCAATourneySeedRoundSlots.csv
 40K -rw-r--r-- 1 nobody  37K Mar  6 18:06 MNCAATourneySeeds.csv
 48K -rw-r--r-- 1 nobody  48K Mar  6 18:06 MNCAATourneySlots.csv
5.3M -rw-r--r-- 1 nobody 5.3M Mar  6 18:06 MRegularSeasonCompactResults.csv
 11M -rw-r--r-- 1 nobody  11M Mar  6 18:06 MRegularSeasonDetailedResults.csv
4.0K -rw-r--r-- 1 

In [3]:
# Define constants
INPUT_DIR = "../input/march-machine-learning-mania-2024/"
DATA_PATH = INPUT_DIR

# Read datasets
tourney_seeds = pd.read_csv(INPUT_DIR + "2024_tourney_seeds.csv")
mteams = pd.read_csv(INPUT_DIR + "MTeams.csv")
wteams = pd.read_csv(INPUT_DIR + "WTeams.csv")

# Concatenate MTeams and WTeams
mteams["Tournament"] = "M"
wteams["Tournament"] = "W"
teams = pd.concat([mteams, wteams])

# Map TeamID to TeamName
team_map = mteams.set_index("TeamID")["TeamName"]
tourney_seeds["TeamName"] = tourney_seeds["TeamID"].map(team_map)

# Extract Region and ChalkRank from Seed
tourney_seeds["Region"] = tourney_seeds["Seed"].str[0]
tourney_seeds["ChalkRank"] = tourney_seeds["Seed"].str[1:].astype("int")

In [4]:
# Read MNCAATourneySeeds and MNCAATourneySlots datasets
mtourney_seeds = pd.read_csv(INPUT_DIR + "MNCAATourneySeeds.csv")
mtourney_slots = pd.read_csv(INPUT_DIR + "MNCAATourneySlots.csv")

In [5]:
mtourney_seeds

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374
...,...,...,...
2485,2023,Z12,1433
2486,2023,Z13,1233
2487,2023,Z14,1213
2488,2023,Z15,1421


In [6]:
#Data preprocessing
mtourney_slots["Round"] = mtourney_slots["Slot"].str[:2]

In [7]:
mtourney_slots.query('Round == "R4"')

,Season,Slot,StrongSeed,WeakSeed,Round
56,1985,R4W1,R3W1,R3W2,R4
57,1985,R4X1,R3X1,R3X2,R4
58,1985,R4Y1,R3Y1,R3Y2,R4
59,1985,R4Z1,R3Z1,R3Z2,R4
119,1986,R4W1,R3W1,R3W2,R4
...,...,...,...,...,...
2377,2022,R4Z1,R3Z1,R3Z2,R4
2441,2023,R4W1,R3W1,R3W2,R4
2442,2023,R4X1,R3X1,R3X2,R4
2443,2023,R4Y1,R3Y1,R3Y2,R4


In [8]:
mtourney_slots.query("Season == 2023")

,Season,Slot,StrongSeed,WeakSeed,Round
2385,2023,R1W1,W01,W16,R1
2386,2023,R1W2,W02,W15,R1
2387,2023,R1W3,W03,W14,R1
2388,2023,R1W4,W04,W13,R1
2389,2023,R1W5,W05,W12,R1
...,...,...,...,...,...
2447,2023,R6CH,R5WX,R5YZ,R6
2448,2023,W16,W16a,W16b,W1
2449,2023,X16,X16a,X16b,X1
2450,2023,Y11,Y11a,Y11b,Y1


In [9]:
tourney_seeds.query("ChalkRank == 1")

,Tournament,Seed,TeamID,TeamName,Region,ChalkRank
0,M,W01,1345,Purdue,W,1
16,M,X01,1104,Alabama,X,1
32,M,Y01,1222,Houston,Y,1
48,M,Z01,1242,Kansas,Z,1
64,W,W01,3376,NaN,W,1
80,W,X01,3390,NaN,X,1
96,W,Y01,3231,NaN,Y,1
112,W,Z01,3439,NaN,Z,1


In [10]:
mteams.tail(10)

,TeamID,TeamName,FirstD1Season,LastD1Season,Tournament
368,1469,Utah Tech,2021,2024,M
369,1470,Tarleton St,2021,2024,M
370,1471,UC San Diego,2021,2024,M
371,1472,St Thomas MN,2022,2024,M
372,1473,Lindenwood,2023,2024,M
373,1474,Queens NC,2023,2024,M
374,1475,Southern Indiana,2023,2024,M
375,1476,Stonehill,2023,2024,M
376,1477,TX A&M Commerce,2023,2024,M
377,1478,Le Moyne,2024,2024,M


In [11]:
wteams.tail(10)

,TeamID,TeamName,Tournament
366,3469,Utah Tech,W
367,3470,Tarleton St,W
368,3471,UC San Diego,W
369,3472,St Thomas MN,W
370,3473,Lindenwood,W
371,3474,Queens NC,W
372,3475,Southern Indiana,W
373,3476,Stonehill,W
374,3477,TX A&M Commerce,W
375,3478,Le Moyne,W


# Additional preprocessing

In [12]:
tourney_seeds["TeamName"] = tourney_seeds["TeamID"].map(team_map)
tourney_seeds["Region"] = tourney_seeds["Seed"].str[0]

In [13]:
tourney_seeds["SubBracket"] = tourney_seeds["Seed"].str[0]

In [14]:
tourney_seeds[["Region", "SubBracket"]].value_counts()

Region  SubBracket
W       W             32
X       X             32
Y       Y             32
Z       Z             32
Name: count, dtype: int64

In [15]:
tourney_seeds["ChalkRank"] = tourney_seeds["Seed"].str[1:].astype("int")

# Machine Learning Modeling and Feature Engineering

In [16]:
df_seeds = pd.concat([
    pd.read_csv(DATA_PATH + "MNCAATourneySeeds.csv"),
    pd.read_csv(DATA_PATH + "WNCAATourneySeeds.csv"),
    ], ignore_index = True)

df_seeds

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374
...,...,...,...
4093,2023,Z12,3405
4094,2023,Z13,3387
4095,2023,Z14,3241
4096,2023,Z15,3436


In [17]:
df_season_results = pd.concat([
    pd.read_csv(DATA_PATH + "MRegularSeasonCompactResults.csv"),
    pd.read_csv(DATA_PATH + "WRegularSeasonCompactResults.csv"),
], ignore_index = True)

df_season_results.drop(["WLoc", "NumOT"], axis=1, inplace=True)

df_season_results

,Season,DayNum,WTeamID,WScore,LTeamID,LScore
0,1985,20,1228,81,1328,64
1,1985,25,1106,77,1354,70
2,1985,25,1112,63,1223,56
3,1985,25,1165,70,1432,54
4,1985,25,1192,86,1447,74
...,...,...,...,...,...,...
317442,2024,114,3409,76,3396,67
317443,2024,114,3424,63,3129,57
317444,2024,114,3433,69,3348,59
317445,2024,114,3453,70,3236,61


In [18]:
df_season_results.query("Season == 2024")

,Season,DayNum,WTeamID,WScore,LTeamID,LScore
181682,2024,0,1101,64,1329,59
181683,2024,0,1103,81,1355,75
181684,2024,0,1104,105,1287,73
181685,2024,0,1112,122,1288,59
181686,2024,0,1114,71,1402,66
...,...,...,...,...,...,...
317442,2024,114,3409,76,3396,67
317443,2024,114,3424,63,3129,57
317444,2024,114,3433,69,3348,59
317445,2024,114,3453,70,3236,61


In [19]:
teams

,TeamID,TeamName,FirstD1Season,LastD1Season,Tournament
0,1101,Abilene Chr,2014.0,2024.0,M
1,1102,Air Force,1985.0,2024.0,M
2,1103,Akron,1985.0,2024.0,M
3,1104,Alabama,1985.0,2024.0,M
4,1105,Alabama A&M,2000.0,2024.0,M
...,...,...,...,...,...
371,3474,Queens NC,NaN,NaN,W
372,3475,Southern Indiana,NaN,NaN,W
373,3476,Stonehill,NaN,NaN,W
374,3477,TX A&M Commerce,NaN,NaN,W


In [20]:
teams.loc[teams["TeamName"] == "Virginia Tech"]

,TeamID,TeamName,FirstD1Season,LastD1Season,Tournament
338,1439,Virginia Tech,1985.0,2024.0,M
337,3439,Virginia Tech,NaN,NaN,W


In [21]:
df_season_results.query("Season == 2024") \
.query("WTeamID == 3439").shape

(23, 6)

In [22]:
df_season_results.query("Season == 2024") \
.query("LTeamID == 3439")

,Season,DayNum,WTeamID,WScore,LTeamID,LScore
312889,2024,3,3234,80,3439,76
313823,2024,24,3261,82,3439,64
315406,2024,69,3199,89,3439,81
315510,2024,73,3181,63,3439,46


In [23]:
df_season_results

,Season,DayNum,WTeamID,WScore,LTeamID,LScore
0,1985,20,1228,81,1328,64
1,1985,25,1106,77,1354,70
2,1985,25,1112,63,1223,56
3,1985,25,1165,70,1432,54
4,1985,25,1192,86,1447,74
...,...,...,...,...,...,...
317442,2024,114,3409,76,3396,67
317443,2024,114,3424,63,3129,57
317444,2024,114,3433,69,3348,59
317445,2024,114,3453,70,3236,61


In [24]:
df_season_results["ScoreDiff"] = df_season_results["WScore"] - df_season_results["LScore"]

In [25]:
df_season_results = pd.concat([
    pd.read_csv(DATA_PATH + "MRegularSeasonCompactResults.csv").assign(League="M"),
    pd.read_csv(DATA_PATH + "WRegularSeasonCompactResults.csv").assign(League="W"),
], ignore_index = True)

df_season_results.drop(["WLoc", "NumOT"], axis=1, inplace=True)

In [26]:
df_season_results

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,League
0,1985,20,1228,81,1328,64,M
1,1985,25,1106,77,1354,70,M
2,1985,25,1112,63,1223,56,M
3,1985,25,1165,70,1432,54,M
4,1985,25,1192,86,1447,74,M
...,...,...,...,...,...,...,...
317442,2024,114,3409,76,3396,67,W
317443,2024,114,3424,63,3129,57,W
317444,2024,114,3433,69,3348,59,W
317445,2024,114,3453,70,3236,61,W


In [27]:
df_season_results["WTeamName"] = df_season_results["WTeamID"].map(team_map)
df_season_results["LTeamName"] = df_season_results["LTeamID"].map(team_map)

In [28]:
print(df_season_results.columns)


Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'League',
       'WTeamName', 'LTeamName'],
      dtype='object')


In [29]:
df_season_results.sort_values("ScoreDiff", ascending=False)["ScoreDiff"].plot(kind="hist", bins=100)

KeyError: 'ScoreDiff'

In [ ]:
df_season_results.sort_values("ScoreDiff")

In [ ]:
df_season_results.sort_values("ScoreDiff").value_counts()

In [ ]:
# Team Wins and Losses
num_win = df_season_results.groupby(["Season", "WTeamID", "League"]).count()
num_win = num_win.reset_index()[["Season", "WTeamID", "League", "DayNum"]].rename(columns={"DayNum": "NumWins", "WTeamID": "TeamID"})
num_loss = df_season_results.groupby(["Season", "LTeamID", "League"]).count()
num_loss = num_loss.reset_index()[["Season", "LTeamID", "League", "DayNum"]].rename(columns={"DayNum": "NumLosses", "LTeamID": "TeamID"})
team_season_win_loss = num_win.merge(num_loss, on=["Season", "TeamID"], how="outer")
team_season_win_loss["TeamName"] = team_season_win_loss["TeamID"].map(team_map)

# Preprocess Win-Loss data
team_season_win_loss["NumLosses"] = team_season_win_loss["NumLosses"].fillna(0).astype(int)
team_season_win_loss["NumWins"] = team_season_win_loss["NumWins"].fillna(0).astype(int)

In [ ]:
num_win.merge(num_loss, on=["Season", "TeamID"])

In [ ]:
team_season_win_loss["NumLosses"] = team_season_win_loss["NumLosses"].fillna(0).astype("int")
team_season_win_loss["NumWins"] = team_season_win_loss["NumWins"].fillna(0).astype("int")

In [ ]:
# Analyze teams with zero losses in the men's league
print(team_season_win_loss.loc[team_season_win_loss["NumLosses"] == 0].query('League_y == "M"'))

In [ ]:
team_season_win_loss

In [ ]:
df_season_results

In [ ]:
# Compute average score differences
gap_win = df_season_results.groupby(["Season", "WTeamID"])["ScoreDiff"].mean().reset_index().rename(columns={"ScoreDiff": "DiffWins", "WTeamID": "TeamID"})
gap_loss = df_season_results.groupby(["Season", "LTeamID"])["ScoreDiff"].mean().reset_index().rename(columns={"ScoreDiff": "DiffLosses", "LTeamID": "TeamID"})

In [ ]:
# How much points  scored in more average 
gap_win = df_season_results.groupby(["Season", "WTeamID"])["ScoreDiff"].mean().reset_index()
gap_win = gap_win[["Season", "WTeamID", "ScoreDiff"]] \
    .rename(columns={"ScoreDiff": "DiffWins", "WTeamID": "TeamID"})
#How much points scored less in average
gap_loss = df_season_results.groupby(["Season", "LTeamID"])["ScoreDiff"].mean().reset_index()
gap_loss = gap_loss[["Season", "LTeamID", "ScoreDiff"]] \
    .rename(columns={"ScoreDiff": "DiffLosses", "LTeamID": "TeamID"})

In [ ]:
df_season_results

In [ ]:
df_features_season_w = df_season_results.groupby(["Season", "WTeamID", "League"]) \
    .count().reset_index()[["Season", "WTeamID", "League"]].rename(columns={"WTeamID": "TeamID"})
df_features_season_l = df_season_results \
    .groupby(["Season", "LTeamID", "League"]) \
    .count().reset_index()[["Season", "LTeamID", "League"]].rename(columns={"LTeamID": "TeamID"})
df_features_season = pd.concat([df_features_season_w, df_features_season_l], axis=0) \
    .drop_duplicates() \
    .sort_values(["Season", "TeamID", "League"]).reset_index(drop=True)

df_features_season = df_features_season.merge(num_win, on=["Season", "TeamID","League"], how="left")
df_features_season = df_features_season.merge(num_loss, on=["Season", "TeamID", "League"], how="left")
df_features_season = df_features_season.merge(gap_win, on=["Season", "TeamID"], how="left")
df_features_season = df_features_season.merge(gap_loss, on=["Season", "TeamID"], how="left")

df_features_season = df_features_season.fillna(0)

df_features_season[["NumWins", "NumLosses"]] = df_features_season[["NumWins", "NumLosses"]].astype("int")

In [ ]:
df_features_season_l

In [ ]:
df_features_season_w

In [ ]:
df_features_season["WinRatio"] = \
    df_features_season["NumWins"] / \
    (df_features_season["NumWins"] + df_features_season["NumLosses"])
df_features_season["ScoreDiffAvg"] = (
    (df_features_season["NumWins"] * df_features_season["DiffWins"] - 
    df_features_season["NumLosses"] *  df_features_season["DiffLosses"])
    / (df_features_season["NumWins"] * df_features_season["NumLosses"])
)

In [ ]:
FEATURES = ["WinRatio", "ScoreDiffAvg"]

In [ ]:
df_tourney_results = pd.concat([
    pd.read_csv(DATA_PATH + "MNCAATourneyCompactResults.csv"),
    pd.read_csv(DATA_PATH + "WNCAATourneyCompactResults.csv")
], ignore_index=True)
df_tourney_results.drop(["WLoc", "NumOT"], axis=1, inplace=True)

In [ ]:
df_tourney_results

In [ ]:
df = df_tourney_results.copy()
df = df[df["Season"] >= 2016].reset_index(drop=True)

df.head()

In [ ]:
# Merge df with df_seeds based on Season and WTeamID
df = df.merge(
    df_seeds,
    how="left",
    left_on=["Season", "WTeamID"],
    right_on=["Season", "TeamID"]
).drop("TeamID", axis=1).rename(columns={"Seed": "SeedW"})

# Merge df with df_seeds based on Season and LTeamID
df = df.merge(
    df_seeds,
    how="left",
    left_on=["Season", "LTeamID"],
    right_on=["Season", "TeamID"]
).drop("TeamID", axis=1).rename(columns={"Seed": "SeedL"})

# Drop duplicated columns SeedW and SeedL
df = df.loc[:,~df.columns.duplicated()]

In [ ]:
df

In [ ]:
import re

def treat_seed(seed):
    # Remove non-numeric characters from the seed string
    seed = re.sub("[^0-9]", "", seed)
    # Convert the cleaned seed string to an integer
    return int(seed)


In [ ]:
df["SeedW"] = df["SeedW"].apply(treat_seed)
df["SeedL"] = df["SeedL"].apply(treat_seed)

In [ ]:
# Find the unique columns by checking if the column name appears only once
unique_columns = df.columns[df.columns.duplicated(keep=False)]

# Filter out the unique columns and keep only one occurrence of each
df = df.loc[:, ~df.columns.isin(unique_columns)]

# Display the DataFrame after removing duplicated columns
print(df)


In [ ]:
# Merge df with df_features_season based on Season and WTeamID
df = df.merge(
    df_features_season,
    how="left",
    left_on=["Season", "WTeamID"],
    right_on=["Season", "TeamID"]
).rename(columns={
    "NumWins": "NumWinsW",
    "NumLosses": "NumLossesW",
    "DiffWins": "DiffWinsW",
    "DiffLosses": "DiffLossesW",
    "WinRatio": "WinRatioW",
    "ScoreDiffAvg": "ScoreDiffAvgW",
}).drop(columns="TeamID", axis=1)

In [ ]:
df

In [ ]:
def add_loosing_matches(df):
    # Define 'win_rename' dictionary
    win_rename = {
        "WTeamID": "TeamIDA",
        "WScore": "ScoreA",
        "LTeamID": "TeamIDB",
        "LScore": "ScoreB",
    }
    
    # Update 'win_rename' dictionary with columns ending with 'W'
    win_rename.update({c : c[:-1] + "A" for c in df.columns if c.endswith("W")})
    # Update 'win_rename' dictionary with columns ending with 'L'
    win_rename.update({c : c[:-1] + "B" for c in df.columns if c.endswith("L")})

    # Define 'lose_rename' dictionary
    lose_rename = {
        "WTeamID": "TeamIDA",
        "WScore": "ScoreB",
        "LTeamID": "TeamIDB",
        "LScore": "ScoreA",
    }
    
    # Update 'lose_rename' dictionary with columns ending with 'W'
    lose_rename.update({c : c[:-1] + "B" for c in df.columns if c.endswith("W")})
    # Update 'lose_rename' dictionary with columns ending with 'L'
    lose_rename.update({c : c[:-1] + "A" for c in df.columns if c.endswith("L")})

    # Copy the DataFrame
    win_df = df.copy()
    lose_df = df.copy()

    # Rename columns in win_df and lose_df
    win_df = win_df.rename(columns=win_rename)
    lose_df = lose_df.rename(columns=lose_rename)

    # Concatenate win_df and lose_df along axis=0 to add losing matches
    return pd.concat([win_df, lose_df], axis=0, sort=False)


In [ ]:
df = add_loosing_matches(df)

# Add Some Features about Diff Between Teams

In [ ]:
cols_to_diff = [
    "Seed", "WinRatio", "ScoreDiffAvg", # '538rating'
]

for col in cols_to_diff:
    df[col + "Diff"] = df[col + "A"] - df[col + "B"]

In [ ]:
df

In [ ]:
df["ScoreDiff"] = df["ScoreA"] - df["ScoreB"]
df["WinA"] = (df["ScoreDiff"] > 0).astype(int)

In [ ]:
target_feature = "WinA"
avoid = ["ScoreDiff", "Season", "DayNum", "A_Win"]
features = [col for col in df.columns if col not in avoid]

In [ ]:
features = ["TeamIDA",
           #"ScoreA",
           "TeamIDB",
            #ScoreB,
           "SeedA",
           "SeedB",
           "WinRatioA",
           "GapAvgA",
           "WinRatioB",
           "GapAvgB",
           "SeedDiff",
           "WinRatioDiff",
           "GapAvgDiff"]

In [ ]:
%%time
# Develop a cv loop to avoid leaking data from future tournaments---
def kfold_model(train_df, tst_df):
    csv = []
    preds_test = []
    seasons = train_df["Season"].unique()
    
    for season in seasons[1:]:
        print(f"\nValidating on season {season}")
        X_train = train_df[train_df["season"] < season][features].reset_index(drop = True).copy()
        X_val = train_df[train_df["season"] == season][features].reset_index(drop = True).copy()
        
        y_train = train_df[train_df["season"] < season][features].reset_index(drop = True).copy()
        y_val = train_df[train_df["season"] == season][features].reset_index(drop = True).copy()
        
        tst_dataset = tst_df[features].copy()
        
        scaler = MinMaxScaler()
        scaler.fit(X_train)
        
        X_train = scaler.transform(X_train)
        X_val = scaler.transform(X_val)
        tst_dataset = scaler.transform(tst_dataset)
        
        model = XGBClassifier(n_estimators =1024, random_state = 85)
        model.fit(X_train,y_train,eval_set =[(x_val,y_val)], verbose = 0, early_stopping_round = 128)
        pred = model.predict_proba(tst_dataset)
        
        loss = log_loss(y_val, pred)
        cvs.append(loss)
        
        print(f'\t -> Scored {loss:.4f}')
        print(f'\nLocal Cross Validation Score Is: {np.mean(cvs):.3f}','\n')
        return preds_test

In [ ]:
predictions = kfold_model(df, df_test)